# 目录说明

- **流程处理**: 该文件夹包含处理数据的所有相关文件。
  - **input**: 存放输入数据和权重。
    - **data**: 存放原始数据文件，CSV 格式，命名方式为 `biaoxun_yearmonth.csv`。
    - **key**: 存放key.csv和权重.pkl文件
    - **html**: 存放 HTML 文件。
      - **html_year_month**: 存放根据信息化筛选后，具体到月份的原 HTML 文件。
  - **output**: 存放处理后的输出结果。
    - **result**: 存放处理结果
    - **sup_name**: 存放招标代理机构名称
  - **main.ipynb**: 运行脚本

In [2]:
import pandas as pd
import numpy as np
import re
import csv
import jieba
import joblib
import os
import csv
from bs4 import BeautifulSoup
from collections import Counter
from collections import defaultdict
from numba import prange

# 1.信息化处理

## 1.1关键词匹配算法

In [67]:
input_file = './input/data/chengjiao24_10.csv'
output_file = './output/result/informatization_projects.csv'

- **input_file**: 文件输入路径
- **output_file**: 文件输出路径
#### 功能描述
通过关键词匹配算法，将信息化标讯进行初步筛选，结果保存为 `output_file`。

In [68]:
# 读取原CSV文件  
data = pd.read_csv(input_file, encoding='GBK')  
# 读取关键字  
tkeys = pd.read_excel("./input/key/key_xxh.xlsx")["Tkey"]  
fkeys = pd.read_excel("./input/key/key_xxh.xlsx")["Fkey"]  
  
# 转换为集合以去除重复项  
Tkeyset = set(tkeys)  
Fkeyset = set(fkeys)  
  
# 存储判断为“是”的行  
informatization_rows = []  
  
# 判断逻辑  
for index, row in data.iterrows():  
    project_name = row["采购项目名称"]  
    is_informatization = False  
    for tkey in Tkeyset:  
        if str(tkey) in str(project_name):  
            contains_fkey = False  
            for fkey in Fkeyset:  
                if str(fkey) in str(project_name):  
                    contains_fkey = True  
                    break  
            # 如果包含Tkey但不包含任何Fkey，则认为是信息化项目  
            if not contains_fkey:  
                is_informatization = True  
                break  
    # 如果判断为信息化项目，则添加到informatization_rows列表中  
    if is_informatization:  
        informatization_rows.append(row)  
  
# 将判断为“是”的行（完整行）写入新的CSV文件  
informatization_df = pd.DataFrame(informatization_rows)  
informatization_df.to_csv(output_file, index=False)  
  
print(f"共有 {len(informatization_rows)} 个项目被判断为信息化项目。")

共有 1780 个项目被判断为信息化项目。


## 1.2.机器学习算法

In [69]:
input_file = output_file
output_file = './output/result/informatization_projects.csv'

- **input_file**: 文件输入路径
- **output_file**: 文件输出路径
#### 功能描述
文件输入路径同第1步输出路径,通过关机器学习算法，将信息化标讯进一步筛选，结果保存为 `output_file`。

In [70]:
# 加载模型并进行预测=======================================================
# 读取新的测试数据
new_data = pd.read_csv(input_file)
new_datas = pd.read_csv(input_file)
# 分词
new_datas['采购项目名称'] = new_datas['采购项目名称'].apply(lambda x: ' '.join(jieba.cut(x)))

# 加载保存的模型和向量化器
loaded_model = joblib.load('./input/key/random_forest_model.pkl')
loaded_vectorizer = joblib.load('./input/key/tfidf_vectorizer.pkl')

# 转换新的测试数据
new_X_test_tfidf = loaded_vectorizer.transform(new_datas['采购项目名称'])

# 进行预测
new_predictions = loaded_model.predict(new_X_test_tfidf)

# 将预测结果添加到新的测试数据中
new_data['prediction'] = new_predictions

# 过滤掉 prediction 值为 0 的行，只保留 prediction 值为 1 的行
filtered_data = new_data[new_data['prediction'] == 1]

# 删除 prediction 列
filtered_data = filtered_data.drop(columns=['prediction'])

# 将结果保存回Excel文件
filtered_data.to_csv(output_file, index=False)

print("预测完成，结果已保存到 ", output_file)


预测完成，结果已保存到  ./output/result/informatization_projects.csv


# 2.补供应商

## 2.1提取招标代理机构名称

In [71]:
input_file = output_file
output_file = './output/result/informatization_projects.csv'
output_sup = './output/sup_name/2024_02/sup_name.csv'
folder_path = './input/html/2024_10/'

- **input_file**: 文件输入路径
- **output_file**: 文件输出路径
- **output_sup**: 保存招标代理机构
- **folder_path**: html输入路径
#### 功能描述
从`folder_path`中提取招标代理机构名称，名称保存到`output_sup`

In [33]:
# 创建一个空列表来存储数据
data = []
i = 0
# 遍历文件夹中的 HTML 文件
for filename in os.listdir(folder_path):
    i += 1
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        soup = BeautifulSoup(content, 'html.parser')
        agent_name_td = soup.find('td', text='代理机构名称')
        if agent_name_td:
            agent_name = agent_name_td.find_next_sibling('td')
            if agent_name:
                agent_uuid = os.path.splitext(filename)[0]
                agent_name_text = agent_name.get_text().strip()
                print(agent_uuid, agent_name_text)
                
                # 将数据添加到列表中
                data.append({'UUID': agent_uuid, 'Name': agent_name_text})

# 创建 DataFrame
df = pd.DataFrame(data)

# 将 DataFrame 保存为 CSV 文件
df.to_csv(output_sup, index=False, encoding='utf-8')

print(f"代理机构名称已写入 {output_sup}, 总共{i}个")


t20241001_23295329 中智达项目管理有限公司
t20241001_23295338 北京国际招标有限公司
t20241001_23295343 新余长兴工程咨询有限公司
t20241001_23295346 江西国政招标咨询有限公司
t20241001_23295350 江西如衡工程管理有限公司
t20241001_23295358 江西省机电设备招标有限公司
t20241001_23295362 杭州赛德建设工程管理咨询有限公司
t20241001_23295371 江西诚闳招标代理有限公司
t20241001_23295378 江西省机电设备招标有限公司
t20241001_23295381 九江新亿招标咨询有限公司
t20241001_23295385 江西省机电设备招标有限公司
t20241001_23295391 江西阳晨项目管理咨询有限公司
t20241001_23295392 江西省百巨招标咨询有限公司
t20241001_23295429 江西铭大招标代理咨询有限公司
t20241001_23295431 江西省鼎跃招标咨询有限公司
t20241001_23295434 中航技国际经贸发展有限公司
t20241001_23295454 余姚市卫生健康系统部门集中采购中心
t20241001_23295490 上海市静安区政府采购中心
t20241001_23295502 浙江建杭工程咨询有限公司
t20241001_23295504 上海新建设工程咨询有限公司
t20241001_23295505 浙江建杭工程咨询有限公司
t20241001_23295529 上海市政府采购中心
t20241001_23295543 甘肃河山项目咨询管理有限责任公司
t20241001_23295552 浙江信镧建设工程咨询有限公司
t20241001_23295564 新疆松瑞工程项目管理咨询有限公司
t20241001_23295587 浙江省政府采购中心
t20241001_23295590 浙江省政府采购中心
t20241001_23295599 华诚工程咨询集团有限公司
t20241001_23295619 上海市浦东新区政府采购中心
t20241001_23295620 上海市浦东新区政府采购中心
t20241001_23295621 新

t20241008_23301453 广西同泽工程项目管理股份有限公司
t20241008_23301454 广东和盛招标代理有限公司
t20241008_23301472 广西德元工程项目管理有限责任公司
t20241008_23301488 海南省政府采购中心
t20241008_23301492 福建省中达招标代理有限公司
t20241008_23301502 定西市安投工程咨询集团有限公司
t20241008_23301519 丰宁满族自治县公共资源交易中心
t20241008_23301533 广西恒鑫全过程工程咨询有限公司
t20241008_23301545 远眺（上海）招标服务有限公司
t20241008_23301597 江西省安澜工程咨询有限公司
t20241008_23301627 衢州市开成工程造价咨询事务所有限公司
t20241008_23301631 广西德元工程项目管理有限责任公司
t20241008_23301656 赣州市长诚工程咨询有限公司
t20241008_23301660 广东金扬教育采购中心有限公司
t20241008_23301663 新疆华域建设工程项目管理咨询有限公司
t20241008_23301665 深圳市南科大资产经营管理有限公司
t20241008_23301714 公诚管理咨询有限公司
t20241008_23301753 山东东德项目管理有限公司
t20241008_23301767 广西同泽工程项目管理股份有限公司
t20241008_23301782 重庆鼎创招标代理有限公司
t20241008_23301814 榆林众昌达项目管理有限公司
t20241008_23301856 四川中泽盛世招标代理有限公司
t20241008_23301860 山西兰花真诚招标代理有限公司
t20241008_23301865 华诚博远工程咨询有限公司
t20241008_23301916 山东齐信全程工程咨询有限公司
t20241008_23301925 陕西中冠世纪项目咨询有限公司
t20241008_23301932 陕西中冠世纪项目咨询有限公司
t20241008_23301960 成都公恒信项目管理有限公司
t20241008_23302015 红城国际工程项目管理有限公司
t20241008_23302

t20241008_23308623 大同市知顺招标代理有限公司
t20241008_23308657 嘉信全过程项目咨询管理有限公司
t20241008_23308658 嘉信全过程项目咨询管理有限公司
t20241008_23308666 嘉信全过程项目咨询管理有限公司
t20241008_23308667 嘉信全过程项目咨询管理有限公司
t20241008_23308696 宁波国际投资咨询有限公司
t20241008_23308737 云南冠睿咨询有限公司
t20241008_23308744 中电商务（北京）有限公司
t20241008_23308747 青海仁捷工程项目管理有限公司
t20241008_23308752 吉林省恒睿项目管理有限公司
t20241008_23308760 吉林省恒睿项目管理有限公司
t20241008_23308766 安徽宜城工程咨询有限公司
t20241008_23308772 宁夏万丰招标代理有限公司
t20241008_23308798 云南九政工程管理服务有限公司
t20241008_23308804 海逸恒安项目管理有限公司
t20241008_23308806 海逸恒安项目管理有限公司
t20241008_23308819 广州市政府采购中心
t20241008_23313141 辽宁纵横工程管理有限公司
t20241009_23309072 江西衡正招投标技术咨询有限公司
t20241009_23309116 江西省机电设备招标有限公司
t20241009_23309143 江西省久安招标代理有限公司
t20241009_23309144 江西省博远工程管理有限公司
t20241009_23309150 江西骏马招标咨询有限公司
t20241009_23309151 江西省百巨招标咨询有限公司
t20241009_23309159 赣州市长诚工程咨询有限公司
t20241009_23309162 江西省机电设备招标有限公司
t20241009_23309164 九江市人众招投标咨询有限公司武宁县分公司
t20241009_23309166 江西省久安招标代理有限公司
t20241009_23309175 江西省机电设备招标有限公司
t20241009_23309180 江西省机电设备招标有限公司
t20241

t20241009_23314070 上海勤铭建设项目管理有限公司
t20241009_23314072 吉林省中鼎建设工程项目管理有限公司
t20241009_23314139 上海健生教育配置招标有限公司
t20241009_23314161 天津市政府采购中心
t20241009_23314164 湖北一信思诚项目咨询管理有限公司
t20241009_23314207 浙江天诺招标代理有限公司
t20241009_23314216 浙江金业管理咨询有限公司
t20241009_23314218 大宁县政府采购中心
t20241009_23314221 四川锦创全过程工程项目管理有限公司
t20241009_23314271 中归咨询管理（北京）有限公司
t20241009_23314274 上海财瑞建设管理有限公司
t20241009_23314312 辽宁轩宇工程管理有限公司
t20241009_23314322 西宁深稳工程管理有限公司
t20241009_23314331 国信招标集团股份有限公司
t20241009_23314353 江苏中际招标代理有限公司
t20241009_23314368 连云港市公共资源交易中心
t20241009_23314388 四川省政府采购中心
t20241009_23314402 山东至和招标代理有限公司
t20241009_23314417 雅安乾新招投标代理有限公司
t20241009_23314432 无锡市公共资源交易中心
t20241009_23314441 江苏省设备成套股份有限公司
t20241009_23314444 临沂市财信招标有限责任公司
t20241009_23314497 嘉诚建设管理有限公司
t20241009_23314499 山东泰邦项目管理咨询有限公司
t20241009_23314500 山东众泰工程管理有限公司
t20241009_23314524 山东至和招标代理有限公司
t20241009_23314525 中岐能工程项目管理有限公司
t20241009_23314538 山东聚智技术咨询有限公司
t20241009_23314548 四川众同信工程咨询有限公司
t20241009_23314555 营口市营商环境建设中心（营口市公共资源交易中心）
t20241009_233

t20241009_23319749 大同市晶鹏工程项目管理有限公司
t20241009_23319756 新疆共建恒业信息咨询有限责任公司
t20241009_23319780 克拉玛依市金科工程监理有限责任公司
t20241009_23319804 固原市公共资源交易中心（固原市政府采购中心）
t20241009_23319811 托克逊县政务服务和公共资源交易中心
t20241009_23319828 新疆国际招标中心有限公司
t20241009_23319833 云之龙咨询集团有限公司
t20241009_23319886 江苏易采招标代理有限公司
t20241009_23319924 海逸恒安项目管理有限公司
t20241009_23319934 海逸恒安项目管理有限公司
t20241009_23319969 广东有德招标采购有限公司
t20241009_23320010 中化商务有限公司
t20241009_23320021 固原市公共资源交易中心（固原市政府采购中心）
t20241009_23320031 吉林省东启项目管理有限公司
t20241009_23320039 大连市政府采购中心
t20241009_23320041 宁夏维实工程咨询有限公司
t20241009_23320055 新疆新德旺建设工程项目管理咨询有限公司
t20241009_23320067 新疆世纪星工程咨询有限公司
t20241009_23320117 宁波中基国际招标有限公司
t20241009_23320164 陕西省采购招标有限责任公司
t20241009_23320179 云南谦和工程咨询有限公司
t20241009_23320191 内蒙古汇正项目管理有限公司
t20241009_23320198 云南磊鑫咨询有限公司
t20241009_23320267 山东凯泽项目管理有限公司
t20241009_23320291 北京悦呈工程管理有限公司
t20241009_23320324 盛和招标代理有限公司
t20241009_23320354 海逸恒安项目管理有限公司
t20241009_23320473 德宏州浩创工程管理服务有限公司
t20241009_23320483 鄂尔多斯市政府采购中心
t20241009_23320513 内蒙古东亨项目管理有限公司
t

...

t20241012_23351196 陕西万德招标有限公司
t20241012_23351201 望江县项目咨询管理有限公司
t20241012_23351205 陕西万德招标有限公司
t20241012_23351209 广西海恒项目管理有限公司
t20241012_23351250 锡林郭勒盟公共资源交易中心（锡林郭勒盟政府采购中心）
t20241012_23351257 福建中晖立建设工程管理有限公司
t20241012_23351279 湖州恒合工程管理有限公司
t20241012_23351318 浙江致信招标代理有限公司
t20241012_23351379 深圳市锦泓兴投资有限责任公司
t20241012_23351385 中招国际招标有限公司
t20241012_23351408 云南谦和工程咨询有限公司
t20241012_23351421 宁夏和顺祥项目管理有限公司
t20241012_23351432 海南政杰工程项目管理有限公司
t20241012_23351436 重庆市渝北区公共资源交易中心
t20241012_23351445 中昕国际项目管理有限公司
t20241012_23351488 江西仁义招标咨询有限公司
t20241012_23351494 重庆市政府采购中心
t20241012_23351503 内蒙古自治区公共资源交易中心
t20241012_23351511 望江县项目咨询管理有限公司
t20241012_23351526 乐山市政府采购中心
t20241012_23351537 四川安鑫招投标代理有限公司
t20241012_23351570 浙江豪圣建设项目管理有限公司
t20241012_23351581 深圳交易咨询集团有限公司
t20241012_23351593 四川思渠国际招标有限公司
t20241012_23351617 国家税务总局岳阳市屈原管理区税务局
t20241012_23351619 江苏大成工程咨询有限公司
t20241012_23351621 中化商务有限公司
t20241012_23351642 新疆翔航项目管理有限公司
t20241012_23351661 贵州聚丰华诚项目管理有限公司
t20241012_23351681 深圳市约法招标有限公司
t20241012_23351699 

t20241013_23356096 上海市浦东新区政府采购中心
t20241013_23356102 上海市政府采购中心
t20241013_23356104 上海市政府采购中心
t20241013_23356119 衢州兴浩工程咨询有限公司
t20241013_23356143 浙江鼎泰工程管理有限公司
t20241013_23356150 浙江天诚工程咨询有限公司
t20241013_23356225 浙江国际招投标有限公司
t20241013_23356231 浙江国际招投标有限公司
t20241013_23356251 浙江省国际技术设备招标有限公司
t20241013_23356266 合肥包河公共资源交易有限公司
t20241013_23356286 浙江天平投资咨询有限公司
t20241013_23356320 浙江添翼工程管理有限公司
t20241013_23356351 浙江建安工程管理有限公司
t20241013_23356359 青海诚德工程咨询管理有限公司
t20241013_23356389 华诚工程咨询集团有限公司
t20241013_23356406 杭州市公共资源交易中心临安分中心
t20241013_23356428 台州市公共资源交易中心（台州市政府采购中心）
t20241013_23356432 安徽兴咨工程项目管理有限公司
t20241013_23356479 浙江华夏工程管理有限公司
t20241013_23356489 安徽时宜项目管理有限公司
t20241013_23356505 安徽百士德工程咨询有限公司
t20241013_23356511 武汉中恒众汇项目管理有限公司
t20241013_23356559 安徽京鼎工程管理有限公司
t20241013_23356637 宣城市宣州区公共资源交易有限公司
t20241013_23356692 芜湖中招项目管理有限公司
t20241013_23356716 亳州市公共资源交易中心
t20241013_23356720 亳州市公共资源交易中心
t20241013_23356731 亳州市公共资源交易中心
t20241013_23356765 马鞍山市政府集中采购中心
t20241013_23356776 泰顺县公共资源交易中心
t20241013_23356795 宿州

t20241014_23361936 龙岩市公物采购招标代理有限公司
t20241014_23361940 上海市青浦区政府采购中心
t20241014_23361943 山东天惠兴招标咨询有限公司
t20241014_23361951 中智卓越项目管理有限公司
t20241014_23361952 山西昊欣招标代理有限公司
t20241014_23361993 中技国际招标有限公司
t20241014_23362008 河北安泰招标有限公司
t20241014_23362047 北京华林源工程咨询有限公司
t20241014_23362051 中国仪器进出口集团有限公司
t20241014_23362087 贵州景瑞尚禾工程咨询有限公司
t20241014_23362107 湖北昱鸿信招标咨询有限公司
t20241014_23362109 福建省漳州市鑫鑫项目管理有限公司
t20241014_23362128 江苏永途工程管理有限公司
t20241014_23362156 瑞金市添盛采购代理有限公司
t20241014_23362201 北京华林源工程咨询有限公司
t20241014_23362204 北京华林源工程咨询有限公司
t20241014_23362240 昱丰(天津)项目管理有限公司
t20241014_23362269 山东声远工程咨询有限公司
t20241014_23362280 中国远东国际招标有限公司
t20241014_23362305 山东天惠兴招标咨询有限公司
t20241014_23362332 山东天惠兴招标咨询有限公司
t20241014_23362405 山西众恒工程招标代理有限公司
t20241014_23362421 贵州拓迈电子招投标服务有限公司
t20241014_23362423 贵州瑞世晟龙工程管理有限公司
t20241014_23362436 内蒙古合和项目管理有限责任公司
t20241014_23362441 贵州新众联项目管理咨询有限公司
t20241014_23362499 明诚汇采项目管理有限公司
t20241014_23362512 兰州兴晟工程造价咨询有限责任公司
t20241014_23362519 四川丰和工程管理有限公司
t20241014_23362527 武汉坤达工程造价咨询有限责任公司
t20

t20241015_23367875 耀华建设管理有限公司
t20241015_23367892 中科华水工程管理有限公司
t20241015_23367893 山东诚和项目管理有限公司
t20241015_23367894 山东诚和项目管理有限公司
t20241015_23367899 山东诚和项目管理有限公司
t20241015_23367900 山东诚和项目管理有限公司
t20241015_23367901 山东诚和项目管理有限公司
t20241015_23367906 山东诚和项目管理有限公司
t20241015_23367910 山东省鲁成招标有限公司
t20241015_23367939 北京国际贸易有限公司
t20241015_23367969 福州华腾招标有限公司
t20241015_23368003 中央国家机关政府采购中心
t20241015_23368008 安徽公共资源交易集团项目管理有限公司
t20241015_23368038 浙江五石中正工程咨询有限公司
t20241015_23368119 江苏中博通信有限公司
t20241015_23368129 江苏海外集团国际工程咨询有限公司
t20241015_23368133 中央国家机关政府采购中心
t20241015_23368201 四川南锋工程管理有限公司
t20241015_23368209 中央国家机关政府采购中心
t20241015_23368210 中央国家机关政府采购中心
t20241015_23368211 枝江市政府采购中心
t20241015_23368224 柳州市政府集中采购中心
t20241015_23368302 上海市政府采购中心
t20241015_23368321 广东华伦招标有限公司
t20241015_23368337 中招国际招标有限公司
t20241015_23368345 山东中青汇采招标咨询有限公司
t20241015_23368357 中轩项目管理有限公司
t20241015_23368365 山东中青汇采招标咨询有限公司
t20241015_23368370 详见公告正文
t20241015_23368378 柳州市政府集中采购中心
t20241015_23368388 广东方元招标采购代理有限公司
t20241015_23368399 

t20241015_23372649 上海市政府采购中心
t20241015_23372671 云南招标股份有限公司
t20241015_23372675 中技国际招标有限公司
t20241015_23372677 广东有德招标采购有限公司
t20241015_23372685 广东和盛招标代理有限公司
t20241015_23372698 宁夏金诚信建设工程咨询有限公司
t20241015_23372719 贵州楹信源项目服务有限公司
t20241015_23372729 上海政采项目管理有限公司
t20241015_23372746 中技国际招标有限公司
t20241015_23372751 浙江信镧建设工程咨询有限公司
t20241015_23372753 杭州意信招标代理有限公司
t20241015_23372774 吉林省荣欣项目管理有限公司
t20241015_23372783 大连机械设备成套有限公司
t20241015_23372812 四川成量工程项目管理有限公司
t20241015_23372830 四川国际招标有限责任公司
t20241015_23372841 河北天华工程管理服务有限公司
t20241015_23372859 浙江信镧建设工程咨询有限公司
t20241015_23372896 宁波市国际招标有限公司
t20241015_23372907 四川毅超工程项目管理有限责任公司
t20241015_23372931 贵州建程招标有限公司
t20241015_23372937 中航材国际招标有限公司
t20241015_23372957 详见公告正文
t20241015_23372969 黑龙江中冠项目管理有限公司
t20241015_23372972 辽宁中诺项目管理有限公司
t20241015_23372980 惠州市晟耀工程咨询有限公司
t20241015_23372990 南充市政府采购中心
t20241015_23373133 海南中顺项目管理有限公司
t20241015_23373135 国家税务总局大邑县税务局
t20241015_23373142 北京市石景山区政府采购中心
t20241015_23373144 四川龙尚招标代理有限公司
t20241015_23373215 华诚工程咨询集团有限公司
t20241015_

## 2.2获取所有的中标单位

#### 功能描述
读取`input_file`中的招标单位和`output_sup`中的招标代理机构,`folder_path`中检索出所有的带有公司和联合体后缀的单位，除去招标代理机构和招标单位剩下默认为中标单位，结果保存到`word_dict`

In [72]:
# 代理机构
sup_data = pd.read_csv(output_sup)
sup_uuids = sup_data['UUID']
sup_names = sup_data['Name']

# 采购单位
empty_data = pd.read_csv(input_file)
empty_uuids = empty_data['UUID']
empty_names = empty_data['采购单位']

# 匹配后缀为公司
company_pattern = re.compile(r'.*公司$')  # 只匹配以 公司 结尾的字符串
company_pattern_copy = re.compile(r'.*公司$') # 只匹配以 公司" 结尾的字符串
# 匹配冒号(中文冒号：)之后的内容
pattern = re.compile(r'：(.*)')

all_company_names = []
word_dict = defaultdict(lambda: defaultdict(int))
single_name = []

# 添加联合体的匹配
joint_venture_pattern = re.compile(r'联合体')

for uuid in empty_uuids:
    filename = f"{uuid}.html"
    file_path = os.path.join(folder_path, filename)
    if os.path.exists(file_path):  # 确保文件存在
        # 读取 HTML 文件内容
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # 解析 HTML
        soup = BeautifulSoup(content, 'html.parser')
#         for div in soup.find_all('div'):
#             text = div.get_text(strip=True)
#             if (company_pattern_copy.match(text)):
#                 # 检查是否包含中文冒号，并且冒号后还有内容
#                 if '：' in text:
#                     parts = text.split('：', 1)
#                     if len(parts) > 1 and parts[1].strip():  # 确保冒号后有内容
#                         uuid = os.path.splitext(filename)[0]
#                         all_company_names.append([text, uuid])
#                 else:
#                     uuid = os.path.splitext(filename)[0]
#                     all_company_names.append([text, uuid])
        for p in soup.find_all('p'):
            text = p.get_text(strip=True)
            if (company_pattern.match(text) or joint_venture_pattern.search(text)):
                # 检查是否包含中文冒号，并且冒号后还有内容
                if '：' in text:
                    parts = text.split('：', 1)
                    if len(parts) > 1 and parts[1].strip():  # 确保冒号后有内容
                        uuid = os.path.splitext(filename)[0]
                        all_company_names.append([text, uuid])
                else:
                    uuid = os.path.splitext(filename)[0]
                    all_company_names.append([text, uuid])
                
        # 查找单位名称    
        for td in soup.find_all('td'):
            text = td.get_text(strip=True)
            if (company_pattern.match(text) or joint_venture_pattern.search(text)):
                # 检查是否包含中文冒号，并且冒号后还有内容
                if '：' in text:
                    parts = text.split('：', 1)
                    if len(parts) > 1 and parts[1].strip():  # 确保冒号后有内容
                        uuid = os.path.splitext(filename)[0]
                        all_company_names.append([text, uuid])
                else:
                    uuid = os.path.splitext(filename)[0]
                    all_company_names.append([text, uuid])
# 输出所有匹配的单位名称和对应的文件名
for name, uuid in all_company_names:
    match = pattern.search(name)
    if match:
        name = match.group(1).strip()
    flag_sup = 0
    for sup_uuid in sup_uuids:
        if uuid == sup_uuid:
            sup_name = sup_names[flag_sup]
            if name == sup_name:
                continue
            else:
                flag_empty = 0
                for empty_uuid in empty_uuids:
                    if uuid == empty_uuid:
                        empty_name = empty_names[flag_empty]
                        if name == empty_name:
                            continue
                        else:
                            single_name.append(name)
                            word_dict[uuid][name] += 1
                            print(name, uuid)
        flag_sup += 1


福建拓尔通软件有限公司 t20241015_23377117
福建拓尔通软件有限公司 t20241015_23377117
兰州远腾软件科技有限公司 t20241015_23376948
兰州远腾软件科技有限公司 t20241015_23376948
成都泰盟软件有限公司 t20241015_23376898
深圳市华图测控系统有限公司 t20241015_23376871
四川三鼎航创科技发展有限公司 t20241015_23376853
四川三鼎航创科技发展有限公司 t20241015_23376853
安正网络安全技术有限公司 t20241015_23376656
广东安盾建设集团有限公司 t20241015_23376537
广东安盾建设集团有限公司 t20241015_23376537
广东华序建设有限公司 t20241015_23376537
广东中创建设集团有限公司 t20241015_23376537
广东安盾建设集团有限公司 t20241015_23376537
广东安盾建设集团有限公司 t20241015_23376537
广东华序建设有限公司 t20241015_23376537
广东中创建设集团有限公司 t20241015_23376537
山大地纬软件股份有限公司 t20241015_23376309
山大地纬软件股份有限公司 t20241015_23376309
北京一木一石科技有限公司 t20241015_23376299
北京一木一石科技有限公司 t20241015_23376299
卫宁健康科技集团股份有限公司 t20241015_23376238
卫宁健康科技集团股份有限公司 t20241015_23376238
湖南君领科技有限公司 t20241015_23376145
湖南君领科技有限公司 t20241015_23376145
湖南超能机器人技术有限公司 t20241015_23376145
北京流深数据科技有限公司 t20241015_23376145
湖南君领科技有限公司 t20241015_23376145
济南超级计算中心有限公司 t20241015_23376019
湖南国科超算科技有限公司 t20241015_23376019
河北超星信息技术有限公司 t20241015_23375982
河北超星信息技术有限公司

湖南凯诺云特智能科技有限公司 t20241014_23363673
西昌四联科技发展有限公司 t20241014_23363705
翰蓝蓝智能技术（重庆）有限公司 t20241014_23363603
翰蓝蓝智能技术(重庆)有限公司 t20241014_23363603
重庆千坤科技有限公司 t20241014_23363603
重庆御云科技有限公司 t20241014_23363603
华诺星空技术股份有限公司 t20241014_23363358
华诺星空技术股份有限公司 t20241014_23363358
北京博雅英杰科技股份有限公司 t20241014_23363508
中电万维信息技术有限责任公司 t20241014_23363525
国泰新点软件股份有限公司 t20241014_23363525
甘肃华悦信息系统工程有限责任公司 t20241014_23363525
中国电信股份有限公司兰州分公司 t20241014_23363525
中国移动通信集团甘肃有限公司兰州分公司 t20241014_23363525
中电万维信息技术有限责任公司 t20241014_23363525
国泰新点软件股份有限公司 t20241014_23363525
甘肃华悦信息系统工程有限责任公司 t20241014_23363525
中国电信股份有限公司兰州分公司 t20241014_23363525
中国移动通信集团甘肃有限公司兰州分公司 t20241014_23363525
中电万维信息技术有限责任公司 t20241014_23363525
国泰新点软件股份有限公司 t20241014_23363525
甘肃华悦信息系统工程有限责任公司 t20241014_23363525
中国电信股份有限公司兰州分公司 t20241014_23363525
中国移动通信集团甘肃有限公司兰州分公司 t20241014_23363525
中电万维信息技术有限责任公司 t20241014_23363525
国泰新点软件股份有限公司 t20241014_23363525
甘肃华悦信息系统工程有限责任公司 t20241014_23363525
中国电信股份有限公司兰州分公司 t20241014_23363525
中国移动通信集团甘肃有限公司兰州分公司 t20241014_23363525
北京

云上广西网络科技有限公司 t20241012_23351871
宁德市艾信达信息科技有限公司 t20241012_23352705
宁德市艾信达信息科技有限公司 t20241012_23352705
厦门帝嘉科技股份有限公司 t20241012_23352707
厦门帝嘉科技股份有限公司 t20241012_23352707
云南卡连科技服务有限公司 t20241012_23352707
广东智慧广电物联网科技有限公司 t20241012_23352707
广州市容达信息科技有限公司 t20241012_23352707
厦门帝嘉科技股份有限公司 t20241012_23352707
厦门帝嘉科技股份有限公司 t20241012_23352707
云南卡连科技服务有限公司 t20241012_23352707
广东智慧广电物联网科技有限公司 t20241012_23352707
广州市容达信息科技有限公司 t20241012_23352707
武汉中科坤程信息产业有限公司 t20241012_23351802
中国电信股份有限公司桂林分公司 t20241012_23352711
中国电信股份有限公司桂林分公司 t20241012_23352711
云上广西网络科技有限公司 t20241012_23352589
云上广西网络科技有限公司 t20241012_23352589
中移建设有限公司 t20241012_23352574
成交供应商: 中移建设有限公司 t20241012_23352574
中移建设有限公司 t20241012_23352574
湖南云宁信息科技有限公司 t20241012_23351732
湖南云宁信息科技有限公司 t20241012_23351617
沈阳创海智澜科技有限公司 t20241012_23352532
沈阳创海智澜科技有限公司 t20241012_23352532
成都久信信息技术股份有限公司 t20241012_23351593
成都久信信息技术股份有限公司 t20241012_23351593
福建浦安信息科技有限公司 t20241012_23351445
福建浦安信息科技有限公司 t20241012_23351445
上海市大数据股份有限公司 t20241012_23352366
上海市大数据股份有限公司 t2024101

四川天府浪潮科技有限公司 t20241012_23344913
山丹县鲲鹏广告传媒有限责任公司 t20241011_23344584
山丹县鲲鹏广告传媒有限责任公司 t20241011_23344584
山丹县鲲鹏广告传媒有限责任公司 t20241011_23344584
山丹县鲲鹏广告传媒有限责任公司 t20241011_23344584
湖南聚正信息产业有限公司 t20241011_23344603
湖南聚正信息产业有限公司 t20241011_23344603
湖南新日信息技术有限公司 t20241011_23344603
湖南新云信安电子科技有限公司 t20241011_23344603
湖南聚正信息产业有限公司 t20241011_23344603
湖南聚正信息产业有限公司 t20241011_23344603
同方知网数字出版技术股份有限公司 t20241011_23344500
同方知网数字出版技术股份有限公司 t20241011_23344500
贵阳宏图科技有限公司 t20241011_23344429
贵阳宏图科技有限公司 t20241011_23344429
北京太极信息系统技术有限公司 t20241011_23344386
中国电信股份有限公司南平分公司 t20241011_23344176
南平市解放路106号电信公司 t20241011_23344176
北京玄蓝云创科技有限公司 t20241011_23343822
北京玄蓝云创科技有限公司 t20241011_23343822
四川日报网络传媒发展有限公司 t20241011_23343826
上海锐倾科教设备有限公司 t20241011_23343762
上海锐倾科教设备有限公司 t20241011_23343762
上海锐倾科教设备有限公司 t20241011_23343762
重庆中联信息产业有限责任公司 t20241011_23343632
重庆中联信息产业有限责任公司 t20241011_23343632
天津超星数图信息技术有限公司 t20241011_23343610
天津超星数图信息技术有限公司 t20241011_23343610
山东维平信息安全测评技术有限公司 t20241011_23343827
山东维平信息安全测评技术有限公司 t20241011_233438

广东粤路勘察设计有限公司 t20241010_23330462
广西南宁合慧成工程咨询有限公司 t20241010_23331416
保定思睿企业管理咨询有限公司 t20241010_23329215
保定思睿企业管理咨询有限公司 t20241010_23329215
中国移动通信集团江苏有限公司徐州分公司 t20241010_23329514
昆明平通科技有限公司 t20241010_23329474
中国电信股份有限公司大理分公司 t20241010_23329473
南昌市守正出奇科技有限公司 t20241010_23330648
南昌市守正出奇科技有限公司 t20241010_23330648
江苏百达智慧网络科技有限公司 t20241010_23329855
北京翔睿信诚科技有限公司 t20241010_23329686
江苏中威科技软件系统有限公司 t20241010_23330294
北京鑫鼎昊通交通设备有限公司 t20241010_23329362
北京鑫鼎昊通交通设备有限公司 t20241010_23329362
北京鑫鼎昊通交通设备有限公司 t20241010_23329362
中国电信股份有限公司 t20241010_23330268
南昌盈华科技有限公司 t20241010_23330611
南昌盈华科技有限公司 t20241010_23330611
广州赛宝联睿信息科技有限公司 t20241010_23329216
广州赛宝联睿信息科技有限公司 t20241010_23329216
江苏有线数据网络有限责任公司 t20241010_23329838
武汉吧哒科技股份有限公司 t20241010_23329083
武汉吧哒科技股份有限公司 t20241010_23329083
重庆汇人大数据有限公司 t20241010_23330148
重庆汇人大数据有限公司 t20241010_23330148
广东宜通衡睿科技有限公司 t20241010_23330148
重庆英之豪科技有限公司 t20241010_23330148
合肥国卫软件有限公司 t20241010_23330374
成都智岚蜀云科技有限公司 t20241010_23330068
北京海联捷讯科技股份有限公司 t20241010_23329565
四川恒晟卓贸易有限公司 t202

浪潮卓数大数据产业发展有限公司 t20241009_23313200
浪潮卓数大数据产业发展有限公司 t20241009_23313200
浪潮卓数大数据产业发展有限公司 t20241009_23313200
视联动力信息技术股份有限公司 t20241009_23313776
视联动力信息技术股份有限公司 t20241009_23313776
江苏好用健康科技有限公司 t20241009_23313130
软通动力信息技术（集团）股份有限公司 t20241009_23313618
软通动力信息技术（集团）股份有限公司 t20241009_23313618
厦门市百安康科技有限公司 t20241009_23313560
厦门市百安康科技有限公司 t20241009_23313560
北京智慧星光信息技术股份有限公司 t20241009_23312786
北京智慧星光信息技术股份有限公司 t20241009_23312786
北京智慧星光信息技术股份有限公司 t20241009_23312786
武汉航天远景科技股份有限公司 t20241009_23313485
武汉航天远景科技股份有限公司 t20241009_23313485
北京荣达信通科技有限公司 t20241009_23313392
北京荣达信通科技有限公司 t20241009_23313392
上海浪潮云计算服务有限公司 t20241009_23313326
上海浪潮云计算服务有限公司 t20241009_23313326
海南智博科技有限公司 t20241009_23313335
海南智博科技有限公司 t20241009_23313335
海南正邦信息科技有限公司 t20241009_23313339
海南正邦信息科技有限公司 t20241009_23313339
成都市锐信安信息安全技术有限公司 t20241009_23312744
广州双优信息技术有限公司 t20241009_23312574
广州市三唯电子有限公司 t20241009_23312574
广州双优信息技术有限公司 t20241009_23312574
广州网鸿信息技术有限公司 t20241009_23312574
广州双优信息技术有限公司 t20241009_23312574
广州市三唯电子有限公司 t20241009_23312574

深圳市世纪鑫诚文化发展有限公司 t20241008_23306453
智升科技（广东）有限公司 t20241008_23306453
广东讯飞启明科技发展有限公司 t20241008_23306453
广东讯飞启明科技发展有限公司 t20241008_23306453
深圳市世纪鑫诚文化发展有限公司 t20241008_23306453
智升科技（广东）有限公司 t20241008_23306453
黑龙江申信科技有限公司 t20241008_23305535
黑龙江申信科技有限公司 t20241008_23305535
黑河迪蓝科技有限公司 t20241008_23305535
哈尔滨软云科技有限公司 t20241008_23305535
哈尔滨市朝阳科技开发有限公司 t20241008_23305535
中国电信股份有限公司东莞分公司 t20241008_23305416
中国电信股份有限公司东莞分公司 t20241008_23305416
天讯瑞达通信技术有限公司东莞博路电信分公司 t20241008_23305416
东莞市智慧万家网络信息科技有限公司 t20241008_23305416
广东特控电子实业有限公司 t20241008_23305333
广东特控电子实业有限公司 t20241008_23305333
江苏苏彩信息技术有限公司 t20241008_23305333
北京中科进出口有限责任公司 t20241008_23305986
北京中科进出口有限责任公司 t20241008_23305986
北京中科进出口有限责任公司 t20241008_23305986
北京中科进出口有限责任公司 t20241008_23305986
北京中科进出口有限责任公司 t20241008_23305986
北京中科进出口有限责任公司 t20241008_23305986
福州迅腾信息技术有限公司 t20241008_23305777
福州迅腾信息技术有限公司 t20241008_23305777
心医云计算科技（新疆）有限公司 t20241008_23305772
心医云计算科技（新疆）有限公司 t20241008_23305772
成都合东科技有限公司 t20241008_23305050
北京万博天地网络技术股份有限公司 t20241008_23304877

安徽峰速网络智能科技有限公司 t20241001_23297291
供应商名称: 芜湖和诚教育咨询有限公司 t20241001_23296819
宿州安之信科技有限公司 t20241001_23296746
宿州安之信科技有限公司 t20241001_23296746
中国铁塔股份有限公司安庆市分公司 t20241001_23296756
中国移动通信集团安徽有限公司宣城分公司 t20241001_23296630
中国电信股份有限公司安徽分公司 t20241001_23296584
合肥一新慧思软件开发有限责任公司 t20241001_23296579
1.供应商名称:中国铁塔股份有限公司蚌埠市分公司 t20241001_23296564
中国铁塔股份有限公司宣城市分公司 t20241001_23296401
中国电信股份有限公司淮南分公司 t20241001_23296132
安徽省大禹水利工程科技有限公司 t20241001_23296076
安徽舜维信息科技有限公司 t20241001_23296037
联通（安徽）产业互联网有限公司（牵头方）、江苏未来网络集团有限公司（联合体成员） t20241001_23295858
联通数字科技有限公司安徽省分公司、江苏未来网络集团有限公司 t20241001_23295858
江苏未来网络集团有限公司、机械工业第六设计研究院有限公司 t20241001_23295858
江苏未来网络集团有限公司、中电鸿信信息科技有限公司、南京科远智慧科技集团股份有限公司 t20241001_23295858
中电鸿信信息科技有限公司、江苏未来网络集团有限公司 t20241001_23295858
江苏未来网络集团有限公司、南京邮数 通信息科技有限公司、新华三技术有限公司 t20241001_23295858
联通数字科技有限公司安徽省分公司、江苏未来网络集团有限公司 t20241001_23295858
江苏未来网络集团有限公司、机械工业第六设计研究院有限公司 t20241001_23295858
江苏未来网络集团有限公司、中电鸿信信息科技有限公司、南京科远智慧科技集团股份有限公司 t20241001_23295858
中电鸿信信息科技有限公司、江苏未来网络集团有限公司 t20241001_23295858
江苏未来网络集团有限公司

## 2.3筛选招标单位

#### 功能说明
将招标单位筛选出来，在`output_file`补充中标单位，以便后续人工补充

In [73]:
output_data = pd.read_csv(output_file)

# 遍历结果字典
for uuid, companies in word_dict.items():
    for company, count in companies.items():
        if count == 1:  # 只处理词频为1的公司
            # 找到对应的行
            if uuid in output_data['UUID'].values:
                output_data.loc[output_data['UUID'] == uuid, 'SellerUnit'] = company

# 保存到 Excel 文件
output_data.to_csv(output_file, index=False)


In [74]:
output_data = pd.read_csv(output_file)

# 遍历结果字典
for uuid, companies in word_dict.items():
    # 找到最大词频
    max_count = max(companies.values())
    
    # 找到具有最大词频的公司
    max_companies = [company for company, count in companies.items() if count == max_count]
    
    # 只有当只有一个公司具有最大词频时才更新
    if len(max_companies) == 1:
        company_name = max_companies[0]
        
        # 使用正则表达式提取冒号后面的内容
        match = re.search(r': (.+)', company_name)
        if match:
            company_name = match.group(1)
        
        # 更新数据框
        if uuid in output_data['UUID'].values:
            output_data.loc[output_data['UUID'] == uuid, 'SellerUnit'] = company_name

# 保存到 Excel 文件
output_data.to_csv(output_file, index=False)


In [75]:
# 读取 CSV 文件
df = pd.read_csv(output_file)

# 填充 SellerUnit 的空值
df['SellerUnit'].fillna(df['中标单位'], inplace=True)

# 删除 中标单位 列
df.drop(columns=['中标单位'], inplace=True)

# 重命名 SellerUnit 列为 中标单位
df.rename(columns={'SellerUnit': '中标单位'}, inplace=True)

# 保存修改后的 DataFrame 到新的 CSV 文件
df.to_csv(output_file, index=False)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_28232\1166950138.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SellerUnit'].fillna(df['中标单位'], inplace=True)


# 3.行业判断

In [76]:
input_file = output_file
output_file = './output/result/informatization_projects.csv'

- **input_file**: 文件输入路径
- **output_file**: 文件输出路径
#### 功能描述
匹配采购单位中的关键词判断行业，保存到原文件中，有部分行业缺失需要补全

In [77]:
# 读取标讯文件，生成结果文件
data = pd.read_csv(input_file)
data['行业'] = ''

# 读取关键词表
classification = pd.read_excel("./input/key/key_organization.xlsx")
classification_boss = pd.read_excel("./input/key/key_boss_orangization.xlsx")
unit = classification['行业']
key = np.array(classification['关键词'])
unit_boss = classification_boss['所属行业'].tolist()
name_boss = np.array(classification_boss['疑难杂症单位名称']).tolist()

# 读取标讯文件中的采购单位名称，并转为array，开始进行判断
names = data["采购单位"]
names = np.array(names)

for idx, name in enumerate(names):
    unit_list = []  # 每次循环重新初始化
    flag = 1

    # 强判断
    for nboss in name_boss:
        if '公司' in name:
            unit_list.append('企业')
            flag = 0
            break
        if '国有资产监督管理' in name:
            unit_list.append('国资委')
            flag = 0
            break
        Jundui_all = ["军民融合创新服务中心", "军民融合服务中心", "军民融合发展服务中心", "军民融合发展研究中心"]
        for jundui in Jundui_all:
            if jundui in name:
                unit_list.append('国防')
                flag = 0
                break
        if flag == 0:
            break
        if nboss in name:
            unit_list.append(unit_boss[name_boss.index(nboss)])
            flag = 0
            break

    # 分词判断关键词
    if flag == 1:
        seg_list = jieba.lcut(name, cut_all=True)
        for seg in seg_list:
            for j in range(len(key)):
                if seg in key[j].split('、'):
                    unit_list.append(unit[j])
                    flag = 0
                    break

    # 反向判断
    if flag == 1:
        for u in range(len(key)):
            if flag == 0:
                break
            for k in key[u].split('、'):
                if k in name:
                    unit_list.append(unit[u])
                    flag = 0
                    break

    # 统计出现次数最多的行业
    if unit_list:
        units = Counter(unit_list)
        most_common_unit = units.most_common(1)[0][0]  # 获取出现次数最多的行业
        data.at[idx, '行业'] = most_common_unit  # 将行业结果保存到 DataFrame 中

# 将结果保存回Excel文件
data.to_csv(output_file, index=False)

print("结果已保存到 ", output_file)

结果已保存到  ./output/result/informatization_projects.csv


# 4.地域修正

In [78]:
input_file = output_file
output_file = './output/result/informatization_projects.csv'

- **input_file**: 文件输入路径
- **output_file**: 文件输出路径
#### 功能描述
根据单位名称进行地域修正，错误的行需要人工补充

In [38]:
input_file = './output/result/chengjiao_result24_10.csv'
output_file = './output/result/chengjiao_result24_10.csv'

In [8]:
# 定义地名库
place_names = {
    "北京": "北京市",
    "上海": "上海市",
    "广州": "广州市",
    "深圳": "深圳市",
    "杭州": "杭州市",
    "天津": "天津市",
    "重庆": "重庆市",
    "哈尔滨": "哈尔滨市",
    "沈阳": "沈阳市",
    "大连": "大连市",
    "济南": "济南市",
    "青岛": "青岛市",
    "郑州": "郑州市",
    "武汉": "武汉市",
    "长沙": "长沙市",
    "南昌": "南昌市",
    "合肥": "合肥市",
    "福州": "福州市",
    "海口": "海口市",
    "南宁": "南宁市",
    "贵阳": "贵阳市",
    "昆明": "昆明市",
    "拉萨": "拉萨市",
    "西安": "西安市",
    "兰州": "兰州市",
    "乌鲁木齐": "乌鲁木齐市",
    "西宁": "西宁市",
    "呼和浩特": "呼和浩特市",
    "银川": "银川市",
    "台北": "台北市",  # 台湾
    "香港": "香港特别行政区",  # 香港
    "澳门": "澳门特别行政区"  # 澳门
}

# 读取表格数据
df = pd.read_csv(input_file, encoding='GBK')


# 函数：识别单位名称中的地名
def extract_place_names(unit_name):
    found_places = []
    # 使用 jieba 分词
    words = jieba.lcut(unit_name)  # 分词，返回列表

    for word in words:
        if word in place_names:
            found_places.append(place_names[word])  # 地名加入结果
            break  # 只要找到一个地名就停止检查

    if not found_places:  # 如果没有找到地名
        short_name = unit_name[:3]  # 取前三个字
        if len(short_name) == 3 and (short_name[2] in ["省", "市"]):  # 如果第三个字是省或市
            found_places.append(short_name)  # 添加到结果

    return ' '.join(found_places)  # 返回找到的地名，空格分隔


# 遍历 G 列进行处理，将结果存储到 R 列
df['补充地区用后删除'] = df['采购单位'].apply(extract_place_names)

df.to_csv(output_file, index=False)

print("处理完成，已保存至", output_file)


处理完成，已保存至 ./output/result/quexiang.csv


In [40]:
# 读取 key_area.csv 文件
areas = pd.read_csv('./input/key/key_area.csv')
unit = np.array(areas['单位名称'])
num = np.array(areas["代码"])

# 读取原 CSV 文件
data = pd.read_csv(input_file, encoding='GBK')
places = data['行政区域']

# 创建一个空的列表来存储结果
results = []

# 错误计数
errors = 0

for place in places:
    if place in unit:
        for i in range(len(num)):
            if unit[i] == place:
                results.append(str(num[i]))
                break  # 找到后可以跳出内层循环
    else:
        errors += 1
        print("错误")
        results.append('错误')

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results, columns=["行政区划编码"])

# 将结果作为新列添加到原 DataFrame
data['行政区划编码'] = results_df

# 保存为新的 CSV 文件
data.to_csv(output_file, index=False)

print(errors)


0


In [42]:
# 读取省级行政区划数据
provices = pd.read_csv('./input/key/key_provice.csv')
unit = np.array(provices['区域'])
num = np.array(provices['编码'])

# 读取数据
data = pd.read_csv(input_file)
places = data['行政区划编码']

# 创建一个空的列表来存储结果
results = []

# 遍历行政区划编码
for place in places:
    if place == '错误':
        results.append('错误')
    else:
        found = False
        for i in range(len(unit)):
            if str(num[i]) == str(place)[0:2]:
                results.append(unit[i])  # 直接存储省级行政区划
                found = True
                break
        if not found:
            results.append('未找到')

data['省级行政区划'] = results

data.to_csv(input_file, index=False)


In [14]:
# 读取 CSV 文件
metadata = pd.read_csv(input_file)
UUID = metadata["UUID"]
ares = metadata["省级行政区划"]
ares_key = ["甘肃省", "海南省", "河北省", "江苏省", "上海省", "山东省", "海南省", "安徽省"]

# 创建一个新的列来存储供应商名称
metadata['供应商名称'] = ""

html_folder = './input/html/2024_10'

for i in range(len(metadata)):
    file_path = os.path.join(html_folder, f"{UUID[i]}.html")  # 构建文件路径
    try:
        if os.path.exists(file_path):  # 确保文件存在
            # 读取 HTML 文件内容
            with open(file_path, 'r', encoding='utf-8') as file:
                html = file.read()
                
            html_table = pd.read_html(html)
            print("=============================")
            print(ares[i])
            supplier_name = ""

            # 根据省级行政区划提取供应商名称
            if ares[i] == "河北省":
                a = html_table[1][2][0]
                supplier_name = str(a).split(" ")[1]
            elif ares[i] == "甘肃省":
                supplier_name = html_table[1][0][1]
            elif ares[i] == "河南省":
                supplier_name = html_table[1][9]
            elif ares[i] == "新疆维吾尔自治区":
                supplier_name = html_table[1]
            elif ares[i] == "上海市":
                supplier_name = html_table[1][3][1]
            elif ares[i] == "山东省":
                match = re.search(r'>供应商名称：(.*?)</|>中标人名称：(.*?)</p|>中标单位名称：(.*?)</p|>中标供应商名称：(.*?)</p|>中标供应商：(.*?)</p', str(html))
                if match:
                    supplier_name = max(re.findall(r'[\u4e00-\u9fa5]+', match.group(1)), key=len, default='')
            elif ares[i] == "安徽省":
                match = re.search(r'供应商名称：([^<]+)(?=<br>)', str(html))
                if match:
                    supplier_name = max(re.findall(r'[\u4e00-\u9fa5]+', match.group(1)), key=len, default='')
            else:
                match = re.search(r'>供应商名称：(.*?)</p|>中标人名称：(.*?)</p|>中标单位名称：(.*?)</p|>中标供应商名称：(.*?)</p|>中标供应商：(.*?)</p', str(html))
                if match:
                    supplier_name = max(re.findall(r'[\u4e00-\u9fa5]+', match.group(1)), key=len, default='')

            # 将提取的供应商名称存入 DataFrame
            metadata.at[i, '供应商名称'] = supplier_name

    except Exception as e:
        print(f"Error processing {ares[i]}: {e}")
        metadata.at[i, '供应商名称'] = ""  # 如果发生异常，设置为空值

# 保存结果回原 CSV 文件
metadata.to_csv(input_file, index=False)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a li

山西省
宁夏回族自治区
江苏省
广东省
福建省


C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a li

上海市
Error processing 上海市: 3
山东省
黑龙江省
新疆维吾尔自治区
新疆维吾尔自治区
新疆维吾尔自治区


C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a li

...

浙江省
未找到
浙江省
甘肃省
上海市
Error processing 上海市: 3
浙江省
上海市
Error processing 上海市: 3
浙江省
上海市
Error processing 上海市: 3
浙江省
江西省
江西省
未找到


C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a li

江西省
未找到
未找到
江西省
未找到
江西省


C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_table = pd.read_html(html)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_30276\3579430143.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a li

江西省
未找到
江西省
江西省
江西省
江西省
江西省


In [85]:
data = pd.read_csv(input_file)

data['中标单位'] = data['中标单位'].combine_first(data['供应商名称'])

data.drop(columns=['供应商名称'], inplace=True)

# 保存结果回原 CSV 文件
data.to_csv(input_file, index=False)


In [45]:
# 读取CSV文件  
main_df = pd.read_csv(input_file, encoding='GBK')  

# 读取行业字典  
key_organization_df = pd.read_excel('./input/key/key_organization.xlsx') 
industry_dict = dict(zip(key_organization_df['行业'], key_organization_df['行业']))

# 读取地区编码字典  
key_area_df = pd.read_csv('./input/key/key_area.csv')
area_dict = dict(zip(key_area_df['单位名称'], key_area_df['代码']))
  
# 检查并输出错误信息  
for index, row in main_df.iterrows():  
    # 检查中标单位是否为空  
    if pd.isnull(row['中标单位']):  
        print(f"中标单位为空的行号：{index + 2}")  
      
    # 检查行业是否为空或不在字典中  
    if pd.isnull(row['行业']):  
        print(f"行业为空的行号：{index + 2}")  
    elif row['行业'] not in industry_dict:  
        print(f"第{index + 2}行填写的行业不符合标准")  
      
    # 检查中标金额是否大于10000或等于0  
    if pd.isnull(row['预算金额']):  
        print(f"第{index + 2}列的金额为空") 
        pass  
    elif row['预算金额'] > 10000:  
        print(f"请仔细检查第{index + 2}列的中标金额")  
#     elif row['预算金额'] == 0:  
#         print(f"第{index + 2}列的中标金额为0")  
      
    # 检查行政区划是否能转换为地区编码  
    if pd.isnull(row['行政区域']):  
        # 如果需要，也可以处理行政区划为空的情况  
        pass  
    elif row['行政区域'] not in area_dict:  
        print(f"第{index + 2}行地区错误")  
  
# 注意：这里假设CSV文件中的列名与代码中的列名一致（如'中标单位'、'行业'、'中标金额'、'行政区划'）。  
# 如果列名不同，请相应地修改代码中的列名。

# 6.识别友商

In [46]:
# 读取原数据
df = pd.read_csv(input_file, encoding='GBK')

# 获取 "中标单位" 列并转换为列表
units = df["中标单位"].tolist()

# 定义字典
dic = {
    '中国软件': ['中国软件', '中软', '中资特种物流', '麒麟', '易鲸捷', '达梦'],
    '太极股份': ['太极', '中电科数智', '瑞太', '金蝶', '人大金仓', '慧点', '第十五研究所'],
    '航天信息': ['航天信息', '数字新疆', '爱信', '航天', '航心', '华迪', '大有'],
    '华宇软件': ['华宇', '溯源云', '捷视飞通', '亿信华辰', '联弈', '万户软件'],
    '东软集团': ['东软'],
    '浪潮软件': ['浪潮'],
    '中国航信': ['中航信', '航信', '民航信息', '航翼', '凯亚', '天信达'],
    '中科软': ['中科软', '中科安胜'],
    '中国移动': ['中国移动', '中移通信', '中移'],
    '中国电信': ['中国电信', '东方口岸', '电信', '中电信数智'],
    '中国联通': ['联通', '联合网络'],
    '神州数码': ['神州', '神码'],
    '东华软件': ['东华', '东华医为'],
    '税友软件': ['税友', '永达科技', '永荣电子', '亿企赢']
}

# 创建一个空列表来存储分类结果
classification_results = []

# 遍历原数据中的每个单位
for unit in units:
    exit_loop = False
    for key, values in dic.items():
        for value in values:
            print(value)
            if value in unit:
                classification_results.append(key)
                exit_loop = True
                break
        if exit_loop:
            break
    if not exit_loop:
        classification_results.append('其他')

# 将分类结果作为新列添加到原 DataFrame
df['集成商'] = classification_results

# 保存结果到新的 CSV 文件
df.to_csv(output_file, index=False)

print("分类完成，结果已保存。")

中国软件
中软
中资特种物流
麒麟
易鲸捷
达梦
太极
中电科数智
瑞太
金蝶
人大金仓
慧点
第十五研究所
航天信息
数字新疆
爱信
航天
航心
华迪
大有
华宇
溯源云
捷视飞通
亿信华辰
联弈
万户软件
东软
浪潮
中航信
航信
民航信息
航翼
凯亚
天信达
中科软
中科安胜
中国移动
中移通信
中移
中国电信
东方口岸
电信
中电信数智
联通
联合网络
神州
神码
东华
东华医为
税友
永达科技
永荣电子
亿企赢
中国软件
中软
中资特种物流
麒麟
易鲸捷
达梦
太极
中电科数智
瑞太
金蝶
人大金仓
慧点
第十五研究所
航天信息
数字新疆
爱信
航天
航心
华迪
大有
华宇
溯源云
捷视飞通
亿信华辰
联弈
万户软件
东软
浪潮
中航信
航信
民航信息
航翼
凯亚
天信达
中科软
中科安胜
中国移动
中移通信
中移
中国电信
东方口岸
电信
中电信数智
联通
联合网络
神州
神码
东华
东华医为
税友
永达科技
永荣电子
亿企赢
中国软件
中软
中资特种物流
麒麟
易鲸捷
达梦
太极
中电科数智
瑞太
金蝶
人大金仓
慧点
第十五研究所
航天信息
数字新疆
爱信
航天
航心
华迪
大有
华宇
溯源云
捷视飞通
亿信华辰
联弈
万户软件
东软
浪潮
中航信
航信
民航信息
航翼
凯亚
天信达
中科软
中科安胜
中国移动
中移通信
中移
中国电信
东方口岸
电信
中电信数智
联通
联合网络
神州
神码
东华
东华医为
税友
永达科技
永荣电子
亿企赢
中国软件
中软
中资特种物流
麒麟
易鲸捷
达梦
太极
中电科数智
瑞太
金蝶
人大金仓
慧点
第十五研究所
航天信息
数字新疆
爱信
航天
航心
华迪
大有
华宇
溯源云
捷视飞通
亿信华辰
联弈
万户软件
东软
浪潮
中航信
航信
民航信息
航翼
凯亚
天信达
中科软
中科安胜
中国移动
中移通信
中移
中国电信
东方口岸
电信
中电信数智
联通
联合网络
神州
神码
东华
东华医为
税友
永达科技
永荣电子
亿企赢
中国软件
中软
中资特种物流
麒麟
易鲸捷
达梦
太极
中电科数智
瑞太
金蝶
人大金仓
慧点
第十五研究所
航天信息
数字新疆
爱信
航天
航心
华迪
大有
华宇
溯源云
捷视飞通
亿信华辰
联弈
万户软件
东软
浪潮
中航信
航信
民航信息
航翼
凯亚
天信达
中科软
中科安胜
中国移动
中

# 7.重生成UUID

In [48]:
IDX = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']

data = pd.read_csv(input_file)
data.insert(0, "NUMBER", range(1, len(data) + 1))
# 获取序号和UUID列
nums = data["NUMBER"]
uuids = data["UUID"]

# 创建一个新的列表来存储生成的 UUID
new_uuids = []

# 初始化计数器
a = 0

# 处理 UUID
for i in range(len(uuids)):
    if i == 0:
        new_uuids.append(uuids[i] + "_" + IDX[0])
    else:
        if uuids[i] == uuids[i - 1]:
            a += 1
            if a > 9:
                b = str(a)
            else:
                b = IDX[a]
            new_uuids.append(uuids[i] + "_" + b)
        else:
            new_uuids.append(uuids[i] + "_" + IDX[0])
            a = 0  # 重置计数器

# 将新的 UUID 列替换原 DataFrame 中的 UUID 列
data["UUID"] = new_uuids

# 保存结果回原文件
data.to_csv(input_file, index=False)


# 8.生成最终结果

In [49]:
input_file = output_file
output_file = './output/result/chengjiao_result24_10.csv'

In [53]:
data = pd.read_csv(input_file, encoding='GBK')

data['中标日期'] = pd.to_datetime(data['中标日期'], format='%Y年%m月%d日 %H:%M')

data['中标日期'] = data['中标日期'].dt.strftime('%Y/%m/%d')

data.to_csv(output_file, index=False)

In [54]:
data1 = {col: [] for col in [
    'NUMBER', 'URL', 'UUID', 'FUND_ID', 'GROUP_ID', 'SUBCONTRACT', 
    'PROCUREMENT_FILE', 'PROJECT_NAME', 'WINNING_BIDDER_NAME', 
    'PROCUREMENT_UNIT_ID', 'PROCUREMENT_TYPE1', 'PROCUREMENT_TYPE2', 
    'PROCUREMENT_TYPE3', 'INDUSTRY_TYPE', 'TERRITORIAL', 
    'TERRITORIAL_NUMB', 'TERRITORIAL_ID', 'TRANSACTION_DATE', 
    'BUDGET_NUMB', 'AMOUNT_NUMB', 'WINNING_TRANS'
]}
df = pd.DataFrame(data1)

data = pd.read_csv(input_file)
df['NUMBER'] = data['NUMBER']
df['URL'] = data['Url']
df['UUID'] = data['UUID']
df['FUND_ID'] = data['资金类别']
df['GROUP_ID'] = data['集成商']
df['PROJECT_NAME'] = data['采购项目名称']
df['WINNING_BIDDER_NAME'] = data['中标单位']
df['PROCUREMENT_UNIT_ID'] = data['采购单位']
df['INDUSTRY_TYPE'] = data['行业']
df['TERRITORIAL'] = data['行政区域']
df['TERRITORIAL_NUMB'] = data['行政区划编码']
df['TERRITORIAL_ID'] = data['省级行政区划']
df['TRANSACTION_DATE'] = data['中标日期']
df['AMOUNT_NUMB'] = data['预算金额']
df['WINNING_TRANS'] = data['采购方式']
df.to_csv(output_file, index=False)